<a href="https://colab.research.google.com/github/VarunB1234/Depression-Detection/blob/main/audioCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import gc
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input
from sklearn.metrics import classification_report

# ------------------ Helper Functions ------------------

def makedata(X):
    for i in range(X.shape[0]):
        if X[i, 1] == 0:
            X[i, 0] = 0
            for j in range(7):
                X[i, j+1] = 0
    return X

def Thresholding(Y_pred, threshold):
    Y_pred2 = []
    for i in range(len(Y_pred)):
        if Y_pred[i] < threshold:
            Y_pred2.append(0)
        else:
            Y_pred2.append(1)
    return np.array(Y_pred2)

def upsample(X_train, Y_train):
    X_train_0 = X_train[Y_train == 0]
    X_train_1 = X_train[Y_train == 1]
    Y_train_1 = Y_train[Y_train == 1]
    size = X_train_0.shape[0] - X_train_1.shape[0]

    X_train = list(X_train)
    Y_train = list(Y_train)

    while size > 0:
        size -= 1
        index = np.random.randint(0, X_train_1.shape[0])
        leave_index = np.random.randint(0, len(X_train))
        X_add = X_train_1[index]
        Y_add = Y_train_1[index]
        X_train[leave_index] = X_add
        X_train.append(X_add)
        Y_train[leave_index] = Y_add
        Y_train.append(Y_add)

    return np.array(X_train), np.array(Y_train)

# ------------------ Load Splits ------------------

train = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv', delimiter=',', encoding='utf-8'))[:, 0:2]
test = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/full_test_split.csv', delimiter=',', encoding='utf-8'))[:, 0:2]
dev = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv', delimiter=',', encoding='utf-8'))[:, 0:2]

X_train = []
Y_train = []
X_val = []
Y_val = []

size = 40000

# ------------------ Load Training Data ------------------

for i in range(len(train)):
    data = pd.read_csv(f'/content/drive/My Drive/MCA Dataset/train_data/{int(train[i][0])}_COVAREP.csv', header=None)
    X_temp = makedata(data.values)
    X_temp = X_temp[-size:]
    X_train.append(X_temp)
    Y_train.append(train[i][1])

for i in range(len(test)):
    data = pd.read_csv(f'/content/drive/My Drive/MCA Dataset/test_data/{int(test[i][0])}_COVAREP.csv', header=None)
    X_temp = makedata(data.values)
    X_temp = X_temp[-size:]
    X_train.append(X_temp)
    Y_train.append(test[i][1])

# ------------------ Load Validation Data ------------------

for i in range(len(dev)):
    data = pd.read_csv(f'/content/drive/My Drive/MCA Dataset/dev_data/{int(dev[i][0])}_COVAREP.csv', header=None)
    X_temp = makedata(data.values)
    X_temp = X_temp[-size:]
    X_val.append(X_temp)
    Y_val.append(dev[i][1])

gc.collect()

X_train = np.array(X_train, dtype=np.float32)
Y_train = np.array(Y_train, dtype=int)

X_val = np.array(X_val, dtype=np.float32)
Y_val = np.array(Y_val, dtype=int)

# ------------------ Upsample ------------------

X_train_upsampled, Y_train_upsampled = upsample(X_train, Y_train)

# ------------------ Define CNN ------------------

class CNN_audio:

    def __init__(self):
        self.classifier = Sequential()
        self.classifier.add(Input(shape=(40000, 74)))
        self.classifier.add(Conv1D(60, 10, activation='relu'))
        self.classifier.add(MaxPooling1D(pool_size=3))
        self.classifier.add(Conv1D(30, 5, activation='relu'))
        self.classifier.add(MaxPooling1D(pool_size=3))
        self.classifier.add(Conv1D(15, 5, activation='relu'))
        self.classifier.add(MaxPooling1D(pool_size=3))
        self.classifier.add(Flatten())
        self.classifier.add(Dropout(0.5))
        self.classifier.add(Dense(units=128, activation='relu'))
        self.classifier.add(Dense(units=1, activation='sigmoid'))
        self.classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    def modelFit(self, X, Y, epoch=5):
        self.classifier.fit(X, Y, epochs=epoch, batch_size=8, verbose=1)

    def modelPredict(self, X):
        return self.classifier.predict(X)

# ------------------ Train & Evaluate ------------------

model = CNN_audio()
model.modelFit(X_train_upsampled, Y_train_upsampled, epoch=5)

Y_val_pred = model.modelPredict(X_val)
Y_val_pred_thresh = Thresholding(Y_val_pred, 0.8)

print(classification_report(Y_val, Y_val_pred_thresh))


Mounted at /content/drive
Epoch 1/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 19s 613ms/step - accuracy: 0.6166 - loss: 4.9287
Epoch 2/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 20s 727ms/step - accuracy: 0.8885 - loss: 0.3120
Epoch 3/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 21s 737ms/step - accuracy: 0.9714 - loss: 0.0592
Epoch 4/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 18s 626ms/step - accuracy: 1.0000 - loss: 0.0083
Epoch 5/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 17s 609ms/step - accuracy: 0.9982 - loss: 0.0067
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step
              precision    recall  f1-score   support

           0       0.72      0.78      0.75        23
           1       0.50      0.42      0.45        12

    accuracy                           0.66        35
   macro avg       0.61      0.60      0.60        35
weighted avg       0.64      0.66      0.65        35

